*DOMINO-SEE构建网络的效率测试（ES）*

测试基准：
+ 时空分辨率：1度分辨率（360x180），30年日数据（10957天）
+ 网络构建方法：基于ES的显著性
+ 计算能力：24核双线程 2.4GHz

In [1]:
from importlib import reload
import xarray as xr
import dominosee.es as es
import dominosee.network as net

In [2]:
from dask.distributed import Client, performance_report
client = Client(processes=False)  
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://10.249.84.134:8787/status,
Dashboard: http://10.249.84.134:8787/status,Workers: 1
Total threads: 10,Total memory: 16.00 GiB
Status: running,Using processes: False
Comm: inproc://10.249.84.134/11711/1,Workers: 0
Dashboard: http://10.249.84.134:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: inproc://10.249.84.134/11711/4,Total threads: 10
Dashboard: http://10.249.84.134:50960/status,Memory: 16.00 GiB
Nanny: None,


# Event selection

In [3]:
from dominosee import eventorize

In [4]:
ds = xr.open_dataset("tests/data/era5.reanalysis.spi30d.0p25deg.china.1950-1979.nc").rename({"latitude": "lat", "longitude": "lon", "spi30d": "SPI1"}).chunk({"time": -1, "lat": 50, "lon": 50})
ds

<xarray.Dataset> Size: 3GB
Dimensions:  (time: 10957, lat: 180, lon: 360)
Coordinates:
  * lat      (lat) float64 1kB 54.5 54.25 54.0 53.75 ... 10.5 10.25 10.0 9.75
  * lon      (lon) float64 3kB 60.0 60.25 60.5 60.75 ... 149.0 149.2 149.5 149.8
    number   int64 8B ...
  * time     (time) datetime64[ns] 88kB 1950-01-01 1950-01-02 ... 1979-12-31
Data variables:
    SPI1     (time, lat, lon) float32 3GB dask.array<chunksize=(10957, 50, 50), meta=np.ndarray>

In [5]:
burst = eventorize.get_event(ds.SPI1, -1.5, "below", event_name="drought", select="start", select_period=1)
burst

<xarray.DataArray 'drought' (lat: 180, lon: 360, time: 10957)> Size: 710MB
dask.array<transpose, shape=(180, 360, 10957), dtype=bool, chunksize=(50, 50, 10957), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 1kB 54.5 54.25 54.0 53.75 ... 10.5 10.25 10.0 9.75
  * lon      (lon) float64 3kB 60.0 60.25 60.5 60.75 ... 149.0 149.2 149.5 149.8
    number   int64 8B ...
  * time     (time) datetime64[ns] 88kB 1950-01-01 1950-01-02 ... 1979-12-31
Attributes:
    threshold:      -1.5
    extreme:        below
    long_name:      drought events
    description:    Events with -1.5 below threshold
    event_name:     drought
    select:         start
    select_period:  1

In [6]:
burst_ts = es.get_event_positions(burst)
burst_ts

<xarray.Dataset> Size: 47MB
Dimensions:          (lat: 180, lon: 360, event: 178)
Coordinates:
  * lat              (lat) float64 1kB 54.5 54.25 54.0 53.75 ... 10.25 10.0 9.75
  * lon              (lon) float64 3kB 60.0 60.25 60.5 ... 149.2 149.5 149.8
    number           int64 8B 0
  * event            (event) int64 1kB 0 1 2 3 4 5 6 ... 172 173 174 175 176 177
Data variables:
    event_positions  (lat, lon, event) int32 46MB dask.array<chunksize=(50, 50, 178), meta=np.ndarray>
    event_count      (lat, lon) int64 518kB dask.array<chunksize=(50, 50), meta=np.ndarray>

# Calculate ES on unstacked and stacked data

In [7]:
burst_tsdiff = es.get_event_time_differences(burst_ts.event_positions, burst_ts.event_count)
burst_tsdiff

<xarray.DataArray 'event_positions' (lat: 180, lon: 360, event: 178)> Size: 92MB
dask.array<where, shape=(180, 360, 178), dtype=float64, chunksize=(50, 50, 178), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 1kB 54.5 54.25 54.0 53.75 ... 10.5 10.25 10.0 9.75
  * lon      (lon) float64 3kB 60.0 60.25 60.5 60.75 ... 149.0 149.2 149.5 149.8
    number   int64 8B 0
  * event    (event) int64 1kB 0 1 2 3 4 5 6 7 ... 171 172 173 174 175 176 177
Attributes:
    long_name:    Event Time Differences
    units:        time steps
    description:  Time differences between consecutive events for each locati...

In [8]:
reload(es)
es_da = es.get_event_sync_from_positions(burst_ts.event_positions, burst_ts.event_positions.copy(), tm=30, 
                                         diffsA=burst_tsdiff, diffsB=burst_tsdiff.copy(), 
                                         event_countsA=burst_ts.event_count, event_countsB=burst_ts.event_count)
es_da

/Users/baymin/opt/anaconda3/envs/dominosee-dev/lib/python3.11/site-packages/dask/array/gufunc.py:485: PerformanceWarning: Increasing number of chunks by factor of 232
  tmp = blockwise(


<xarray.DataArray (latA: 180, latB: 180, lonA: 360, lonB: 360)> Size: 4GB
dask.array<transpose, shape=(180, 180, 360, 360), dtype=uint8, chunksize=(7, 7, 360, 360), chunktype=numpy.ndarray>
Coordinates:
  * latA     (latA) float64 1kB 54.5 54.25 54.0 53.75 ... 10.5 10.25 10.0 9.75
  * lonA     (lonA) float64 3kB 60.0 60.25 60.5 60.75 ... 149.2 149.5 149.8
    number   int64 8B 0
  * latB     (latB) float64 1kB 54.5 54.25 54.0 53.75 ... 10.5 10.25 10.0 9.75
  * lonB     (lonB) float64 3kB 60.0 60.25 60.5 60.75 ... 149.2 149.5 149.8
Attributes:
    long_name:     Event Synchronization
    units:         count
    description:   Number of synchronized events between locations A and B
    tau_max:       30
    output_dtype:  <class 'numpy.uint8'>

# Null model

In [9]:
reload(es)
da_cvalues = es.create_null_model_from_indices(ds.time, es_da.attrs["tau_max"], 
                                               max_events=burst_ts.event_count.max().values, significances=[0.05, 0.01, 0.005])
da_cvalues

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


<xarray.DataArray (noeA: 179, noeB: 179, significance: 3)> Size: 769kB
array([[[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
...
        ...,
        [10, 13, 14],
        [10, 13, 14],
        [10, 12, 13]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [10, 13, 14],
        [10, 13, 14],
        [10, 12, 14]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [10, 12, 13],
        [10, 12, 14],
        [10, 12, 13]]], shape=(179, 179, 3))
Coordinates:
  * noeA          (noeA) int64 1kB 0 1 2 3 4 5 6 ... 172 173 174 175 176 177 178
  * noeB          (noeB) int64 1kB 0 1 2 3 4 5 6 ... 172 173 174 175 176 177 178
  * significance  (significance) float64 24B 0.05 0.01 0.005
Attributes:
    description:  Event synchronization null model for pairs of number of events
    tau_max:      30
    max_events:   178
    min_es:       None

In [10]:
temp = da_cvalues.sel(significance=0.005).squeeze()
temp

<xarray.DataArray (noeA: 179, noeB: 179)> Size: 256kB
array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ..., 14, 14, 13],
       [ 0,  0,  0, ..., 14, 14, 14],
       [ 0,  0,  0, ..., 13, 14, 13]], shape=(179, 179))
Coordinates:
  * noeA          (noeA) int64 1kB 0 1 2 3 4 5 6 ... 172 173 174 175 176 177 178
  * noeB          (noeB) int64 1kB 0 1 2 3 4 5 6 ... 172 173 174 175 176 177 178
    significance  float64 8B 0.005
Attributes:
    description:  Event synchronization null model for pairs of number of events
    tau_max:      30
    max_events:   178
    min_es:       None

In [11]:
da_null = es.convert_null_model_for_locations(da_cvalues, burst_ts.event_count, burst_ts.event_count, sig=0.005)
da_null

<xarray.DataArray (latA: 180, lonA: 360, latB: 180, lonB: 360)> Size: 34GB
array([[[[6, 7, 7, ..., 5, 5, 5],
         [6, 6, 6, ..., 6, 6, 6],
         [6, 6, 6, ..., 5, 5, 6],
         ...,
         [3, 3, 4, ..., 5, 5, 5],
         [3, 3, 3, ..., 5, 5, 5],
         [4, 4, 4, ..., 5, 5, 5]],

        [[7, 7, 7, ..., 5, 5, 6],
         [6, 7, 7, ..., 5, 6, 6],
         [6, 6, 6, ..., 5, 6, 7],
         ...,
         [3, 3, 4, ..., 5, 5, 5],
         [4, 3, 3, ..., 5, 5, 5],
         [4, 4, 4, ..., 4, 4, 5]],

        [[7, 7, 7, ..., 5, 5, 6],
         [6, 7, 7, ..., 5, 6, 6],
         [6, 6, 6, ..., 5, 6, 7],
         ...,
...
         ...,
         [2, 2, 2, ..., 3, 3, 4],
         [3, 2, 2, ..., 3, 3, 3],
         [3, 3, 3, ..., 3, 3, 3]],

        [[5, 4, 4, ..., 4, 4, 4],
         [5, 5, 5, ..., 4, 4, 4],
         [4, 5, 4, ..., 4, 4, 4],
         ...,
         [2, 2, 2, ..., 3, 3, 4],
         [3, 2, 2, ..., 3, 3, 3],
         [3, 3, 3, ..., 3, 3, 3]],

        [[5, 5, 5, ..., 4, 4, 4],
         [5, 5, 5, ..., 4, 4, 4],
         [5, 5, 5, ..., 4, 4, 5],
         ...,
         [2, 2, 2, ..., 3, 3, 4],
         [3, 3, 3, ..., 3, 3, 3],
         [3, 3, 3, ..., 3, 3, 3]]]], shape=(180, 360, 180, 360))
Coordinates:
    noeA          (latA, lonA) int64 518kB 100 105 105 101 90 ... 58 57 57 57 60
    noeB          (latB, lonB) int64 518kB 100 105 105 101 90 ... 58 57 57 57 60
    significance  float64 8B 0.005
  * latA          (latA) float64 1kB 54.5 54.25 54.0 53.75 ... 10.25 10.0 9.75
  * lonA          (lonA) float64 3kB 60.0 60.25 60.5 60.75 ... 149.2 149.5 149.8
    number        int64 8B 0
  * latB          (latB) float64 1kB 54.5 54.25 54.0 53.75 ... 10.25 10.0 9.75
  * lonB          (lonB) float64 3kB 60.0 60.25 60.5 60.75 ... 149.2 149.5 149.8
Attributes:
    description:  Event synchronization null model for pairs of locations
    tau_max:      30
    max_events:   178
    min_es:       None

# get link

In [12]:
es_da = es_da.compute()
es_da

<xarray.DataArray (latA: 180, latB: 180, lonA: 360, lonB: 360)> Size: 4GB
array([[[[ 98,  81,  72, ...,  12,  13,  15],
         [ 81, 103,  82, ...,  12,  14,  14],
         [ 72,  82, 103, ...,   9,   9,  10],
         ...,
         [ 12,  12,   9, ...,  70,  56,  51],
         [ 12,  13,   9, ...,  57,  72,  59],
         [ 14,  13,  10, ...,  52,  60,  77]],

        [[ 75,  72,  66, ...,  12,  15,  11],
         [ 75,  75,  75, ...,  12,  14,  14],
         [ 72,  76,  81, ...,   8,   6,   7],
         ...,
         [ 12,  10,   9, ...,  59,  55,  53],
         [ 13,  10,  11, ...,  55,  55,  59],
         [ 14,  12,  11, ...,  54,  57,  61]],

        [[ 63,  66,  60, ...,  13,  12,  11],
         [ 63,  68,  66, ...,  12,  13,  15],
         [ 61,  69,  67, ...,   7,   7,   8],
         ...,
...
         [  6,   4,   7, ...,  35,  38,  36],
         [  5,   2,   5, ...,  33,  37,  36],
         [  5,   2,   5, ...,  34,  39,  41]],

        [[ 23,  25,  22, ...,   4,   6,   5],
         [ 26,  28,  25, ...,   5,   7,   6],
         [ 22,  24,  22, ...,   5,   7,   6],
         ...,
         [  7,   8,   8, ...,  42,  39,  36],
         [  6,   6,   6, ...,  42,  40,  36],
         [  6,   6,   6, ...,  39,  43,  42]],

        [[ 33,  28,  21, ...,   8,   6,   6],
         [ 28,  37,  26, ...,   9,   7,   7],
         [ 21,  26,  33, ...,   9,   7,   7],
         ...,
         [  8,   9,   9, ...,  55,  48,  47],
         [  6,   7,   7, ...,  48,  55,  49],
         [  6,   7,   7, ...,  47,  49,  58]]]],
      shape=(180, 180, 360, 360), dtype=uint8)
Coordinates:
  * latA     (latA) float64 1kB 54.5 54.25 54.0 53.75 ... 10.5 10.25 10.0 9.75
  * lonA     (lonA) float64 3kB 60.0 60.25 60.5 60.75 ... 149.2 149.5 149.8
    number   int64 8B 0
  * latB     (latB) float64 1kB 54.5 54.25 54.0 53.75 ... 10.5 10.25 10.0 9.75
  * lonB     (lonB) float64 3kB 60.0 60.25 60.5 60.75 ... 149.2 149.5 149.8
Attributes:
    long_name:     Event Synchronization
    units:         count
    description:   Number of synchronized events between locations A and B
    tau_max:       30
    output_dtype:  <class 'numpy.uint8'>

In [13]:
da_link = net.get_link_from_critical_values(es_da, da_null, rule="greater")
da_link

<xarray.DataArray (latA: 180, latB: 180, lonA: 360, lonB: 360)> Size: 4GB
array([[[[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True, False,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
...
         [ True,  True,  True, ...,  True,  True,  True],
         [ True, False,  True, ...,  True,  True,  True],
         [ True, False,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]]]],
      shape=(180, 180, 360, 360))
Coordinates:
  * latA          (latA) float64 1kB 54.5 54.25 54.0 53.75 ... 10.25 10.0 9.75
  * lonA          (lonA) float64 3kB 60.0 60.25 60.5 60.75 ... 149.2 149.5 149.8
    number        int64 8B 0
  * latB          (latB) float64 1kB 54.5 54.25 54.0 53.75 ... 10.25 10.0 9.75
  * lonB          (lonB) float64 3kB 60.0 60.25 60.5 60.75 ... 149.2 149.5 149.8
    noeA          (latA, lonA) int64 518kB 100 105 105 101 90 ... 58 57 57 57 60
    noeB          (latB, lonB) int64 518kB 100 105 105 101 90 ... 58 57 57 57 60
    significance  float64 8B 0.005

In [14]:
da_link.sum()/da_link.size

<xarray.DataArray ()> Size: 8B
array(0.89961521)
Coordinates:
    number        int64 8B 0
    significance  float64 8B 0.005